
This code is mainly taken from:

https://github.com/danielegrattarola/spektral/blob/master/examples/other/graph_signal_classification_mnist.py


In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install spektral

In [ ]:
#Refer to an example, https://www.kaggle.com/delai50/ingv-imaging-ts-for-mt-doom-eruption-prediction
!pip install pyts   
from pyts.image import GramianAngularField

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
       os.path.join(dirname, filename)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import gc
gc.enable()
from tqdm.notebook import tqdm


import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.losses import MeanAbsoluteError
#from tensorflow.keras.metrics import MeanAbsoluteError

from spektral.data import PackedBatchLoader
from spektral.data import Dataset, Graph, DisjointLoader 
from spektral.layers import GCNConv
from spektral.layers.ops import sp_matrix_to_sp_tensor

import scipy.sparse as sp
from sklearn.neighbors import kneighbors_graph
from sklearn.metrics import mean_absolute_error, r2_score

import scipy as scp  
from scipy.signal import butter,filtfilt,freqz
import scipy.fftpack

In [ ]:
# Detect hardware, return appropriate distribution strategy
def get_strategy():
    gpu = ""
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print('Running on TPU ', tpu.master())     
    except ValueError:
        tpu = None
        os.environ["CUDA_VISIBLE_DEVICES"] = "0"
        gpu = tf.config.list_physical_devices("GPU")
        if len(gpu) == 1:
            print('Running on GPU ', gpu)
    if tpu:
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
    elif len(gpu) == 1:
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
        tf.config.optimizer.set_experimental_options({"auto_mixed_precision":True})
    else:
        strategy = tf.distribute.get_strategy()
    print("REPLICAS: ", strategy.num_replicas_in_sync)
    return strategy 

strategy = get_strategy()

In [ ]:
# GLOBAL VARIABLES
PATH = '/kaggle/input/predict-volcanic-eruptions-ingv-oe'

#IMG_SIZE=10 is corr among sensors/10min, 20=corr among sensors/5min, else=GramianAngularField
IMG_SIZE = 10          #10, 20, 28
kk= int(0.3*IMG_SIZE)  #number of neighbours of each node.

# Parameters
CHANNELS = 1 
batch_size = 128  
epochs = 1500        # Number of training epochs
patience = epochs    # Patience for early stopping
l2_reg = 5e-4        # Regularization rate for l2

#BPF
low_cutoff = 1       #Hz, to remove dc
high_cutoff = 35     #Hz
order_bpf = 6        #filter order
fs = 100             #sample rate, 100 Hz
nyq = 0.5 * fs       #Nyquist Frequency

In [ ]:
DEBUG = True   

if DEBUG:
    batch_size = 32  
    epochs = 1500

In [ ]:
def butter_bandpass_filter(data, cutoff_low, cutoff_high, fs, order):
    normal_cutoff_low = cutoff_low / nyq
    normal_cutoff_high = cutoff_high / nyq    
    # Get the filter coefficients 
    b, a = butter(order, [normal_cutoff_low,normal_cutoff_high], btype='band', analog=False)
    y = filtfilt(b, a, data)
    return y

In [ ]:
# HELPING FUNCTIONS
LIMIT_PRECISION = 2**15-1     #16 bits precision on sensors
MINUTES_10 = 60000            #60k=10min
MINUTES_5  = 30000
SPLIT_MINUTES = int(MINUTES_10/MINUTES_5)

def tseries_to_imgAvg(df_list, type_d, method='difference'):   
    df_list.index = df_list['segment_id']
    df_img = np.zeros((df_list.shape[0], IMG_SIZE, IMG_SIZE*CHANNELS, 1))
    
    cols = ['s'+str(i)+str(j) for i in range(10) for j in range(SPLIT_MINUTES)]
    
    for r,seg in enumerate(tqdm(df_list['segment_id'].values.tolist())):
        seg_df = pd.read_csv(os.path.join(PATH, type_d, str(seg)+'.csv'))
        seg_df = seg_df.iloc[:MINUTES_10,:]
        df_rand = pd.DataFrame(np.random.randn(seg_df.shape[0],seg_df.shape[1]), 
                               columns=seg_df.columns, index=seg_df.index)/LIMIT_PRECISION
        #seg_df = seg_df.fillna(0)                      
        seg_df[pd.isna(seg_df)] = df_rand[pd.isna(seg_df)]               
        
        #BPF filtering
        seg_columns = seg_df.columns
        for i in seg_columns:
            seg_df[i] = butter_bandpass_filter(seg_df[i].values, low_cutoff, high_cutoff, fs, order_bpf) 
         
        if IMG_SIZE == 10:     #corr sensors/10min
            A = seg_df.corr()
            A = A.fillna(0)    #NaN if sensor =0
            A[np.abs(A)<0.01]=0
            np.fill_diagonal(A.values, 1)    #missing sensor
            df_img[r, :, 0:IMG_SIZE, 0] = A  #(10,10) 
        elif IMG_SIZE == 20:    #corr sensors/5min
            np_seg = seg_df.values
            seg_sens = np_seg.reshape([MINUTES_5,-1],order='F')  #(30k, 20)
            A = pd.DataFrame(seg_sens, columns=cols)
            A = A.corr()
            A = A.fillna(0)
            A[np.abs(A)<0.01] = 0
            np.fill_diagonal(A.values, 1)    #missing sensor
            df_img[r, :, 0:IMG_SIZE, 0] = A  #(20,20)
        else:       #GramianAngularField
            #norm
            train_input_mean = seg_df.mean()
            train_input_sigma = seg_df.std()
            seg_df_avg = (seg_df - train_input_mean) / train_input_sigma
            seg_df_avg = seg_df_avg.fillna(0)
            seg_df_avg['avg'] = seg_df_avg.mean(axis=1)           
            seg_sens = seg_df_avg['avg'].values.reshape(1,-1)  #(1, 60000)              

            gadf = GramianAngularField(image_size=IMG_SIZE, method=method)
            seg_sens_gadf = gadf.fit_transform(seg_sens)
            df_img[r, :, 0:IMG_SIZE, 0] = seg_sens_gadf[0,:,:]  
            
    return df_img

In [ ]:
#BandPass Filter Frequency Response

b2, a2 = butter(order_bpf, [low_cutoff/nyq, high_cutoff/nyq], btype='band', analog=False)
w2,h2 = freqz(b2,a2, fs=fs)
print(len(w2))  #len(w2) = 50Hz

plt.figure(figsize=(18,4))
plt.subplot(131)
plt.plot(w2, 20 * np.log10(abs(h2)+2**-31), 'b')  #add 2**-31 to avoid log(0)
plt.ylabel('Amplitude [dB]', color='b')
plt.xlabel('Frequency [Hz]')
plt.grid()
plt.title('BPF, passband @ 0.01Hz~35Hz')

bins=20   # 1.9Hz = 50* 20/len(w2)
plt.subplot(132)
plt.plot(w2[:bins], 20 * np.log10(abs(h2[:bins])+2**-31), 'b')
plt.ylabel('Amplitude [dB]', color='b')
plt.xlabel('Frequency [Hz]')
plt.grid()
plt.title('Zoom-In BPF')

bins=256  # 25Hz = 50* 256/len(w2)
plt.subplot(133)
plt.plot(w2[-bins:], 20 * np.log10(abs(h2[-bins:])+2**-31), 'b')
plt.ylabel('Amplitude [dB]', color='b')
plt.xlabel('Frequency [Hz]')
plt.grid()
plt.title('Zoom-In BPF')

In [ ]:
train_df = pd.read_csv('../input/predict-volcanic-eruptions-ingv-oe/train/601524801.csv')  #shortest erupttime=6250
train_df.head(5)

In [ ]:
#add random noise to NaN
dt = train_df.copy()
df_rand = pd.DataFrame(np.random.randn(dt.shape[0],dt.shape[1]), columns=dt.columns, index=dt.index)/(2**15-1)
dt[pd.isna(dt)] = df_rand[pd.isna(dt)]

In [ ]:
#Original Sensors
fig, ax = plt.subplots(nrows=10, ncols=2, figsize=(24, 30))
fig.subplots_adjust(hspace = .5)
colors = plt.rcParams["axes.prop_cycle"]()
  
# Sensor#1 ~ #10
for j in range(1,11):
        
    fft = np.fft.fft(dt[f'sensor_{j}'].values)
    psd = np.abs(fft) ** 2
    fftfreq = scp.fftpack.fftfreq(len(psd),1/fs)
    i = fftfreq > 0

    c = next(colors)["color"]
    ax[j-1,0].plot(dt[f'sensor_{j}'].values,color=c)
    ax[j-1,0].set_title('Sensor_'+str(j))
    ax[j-1,0].set_xlabel('time-domain samples')
    ax[j-1,0].set_ylabel('Amp')

    ax[j-1,1].plot(fftfreq[i], 10 * np.log10(psd[i]),color=c)
    ax[j-1,1].set_title('Sensor_'+str(j))
    ax[j-1,1].set_xlabel('Frequency (Hz)')
    ax[j-1,1].set_ylabel('PSD (dB)')


Sensor 4:Very strong low frequency signal close to DC. This created a non-stationary bias. Strong DC at sensors 1,4,6,7,9,10

Sensor 1,5,7,9,10: Impulsive signals

Sensor 2,3,8: Small random noises

Sensor 9: Peak PSD 125dB, noise floor PSD 50dB, dynamic range of sensor 125-50 = 75dB about 13bits precision

In [ ]:
#BPF on all sensors to remove DC
seg_columns = dt.columns
for i in seg_columns:
    dt[i] = butter_bandpass_filter(dt[i].values, low_cutoff, high_cutoff, fs, order_bpf) 

In [ ]:
#norm
dt_mean = dt.mean()
dt_sigma = dt.std()
dt_avg = (dt - dt_mean) / dt_sigma
#avg all sensors
dt_avg['avg'] = dt_avg.mean(axis=1)  

In [ ]:
fig, ax = plt.subplots(nrows=11, ncols=2, figsize=(24, 30))
fig.subplots_adjust(hspace = .5)
colors = plt.rcParams["axes.prop_cycle"]()
  
# Sensor#1 ~ #10
for j in range(1,11):
        
    fft = np.fft.fft(dt_avg[f'sensor_{j}'].values)
    psd = np.abs(fft) ** 2
    fftfreq = scp.fftpack.fftfreq(len(psd),1/fs)
    i = fftfreq > 0

    c = next(colors)["color"]
    ax[j-1,0].plot(dt_avg[f'sensor_{j}'].values,color=c)
    ax[j-1,0].set_title('Sensor_'+str(j))
    ax[j-1,0].set_xlabel('time-domain samples')
    ax[j-1,0].set_ylabel('Amp')

    ax[j-1,1].plot(fftfreq[i], 10 * np.log10(psd[i]),color=c)
    ax[j-1,1].set_title('Sensor_'+str(j))
    ax[j-1,1].set_xlabel('Frequency (Hz)')
    ax[j-1,1].set_ylabel('PSD (dB)')

#Average Sensor signals
j +=1
fft = np.fft.fft(dt_avg.avg)
psd = np.abs(fft) ** 2
fftfreq = scp.fftpack.fftfreq(len(psd),1/fs)
i = fftfreq > 0

c = next(colors)["color"]
ax[j-1,0].plot(dt_avg.avg,color=c)
ax[j-1,0].set_title('Sensor_Avg')
ax[j-1,0].set_xlabel('time-domain samples')
ax[j-1,0].set_ylabel('Amp')

ax[j-1,1].plot(fftfreq[i], 10 * np.log10(psd[i]),color=c)
ax[j-1,1].set_title('Sensor_Avg')
ax[j-1,1].set_xlabel('Frequency (Hz)')
ax[j-1,1].set_ylabel('PSD (dB)')    


Freq below 1Hz are removed.  Very obvious in sensors 4 time-domain signal.

Sensors 2,3,8 noises are amplified due to normalization.  Not sure this is a good idea.  For correlations usage, correlations with random noise should resulted in zero, this should be fine.  Another way is to average the sensors first, then normalize.

In [ ]:
#Lets look at sensor 5 on every minute.

fig, ax = plt.subplots(nrows=10, ncols=2, figsize=(24, 30))
fig.subplots_adjust(hspace = .5)
colors = plt.rcParams["axes.prop_cycle"]()
  
# Minute#1 ~ #10 on sensor 5
for j in range(10):
        
    fft = np.fft.fft(dt_avg.sensor_5[6000*(j):6000*(j+1)])
    psd = np.abs(fft) ** 2
    fftfreq = scp.fftpack.fftfreq(len(psd),1/fs)
    i = fftfreq > 0

    c = next(colors)["color"]
    ax[j,0].plot(dt_avg.sensor_5[6000*(j):6000*(j+1)],color=c)
    ax[j,0].set_title('Minute'+str(j+1))
    ax[j,0].set_xlabel('time-domain samples')
    ax[j,0].set_ylabel('Amp')

    ax[j,1].plot(fftfreq[i], 10 * np.log10(psd[i]),color=c)
    ax[j,1].set_title('Minute'+str(j+1))
    ax[j,1].set_xlabel('Frequency (Hz)')
    ax[j,1].set_ylabel('PSD (dB)')

In [ ]:
del train_df,dt,dt_avg
gc.collect()

In [ ]:
# Load data
train_df = pd.read_csv(os.path.join(PATH,'train.csv'))
sub = pd.read_csv(os.path.join(PATH,'sample_submission.csv'))

In [ ]:
train_df.head(3)

In [ ]:
df_list_train = train_df.copy()
df_list_test = sub.copy()

if DEBUG:
    df_list_train = df_list_train[df_list_train["time_to_eruption"] < 600_000]  #debug
    df_list_test = df_list_test[:32]  #debug
    
df_list_train.shape, df_list_test.shape

In [ ]:
# Prepare data
X_train = tseries_to_imgAvg(df_list_train, type_d='train', method='difference')
y_train = df_list_train['time_to_eruption'].values.reshape(-1,1)
X_test = tseries_to_imgAvg(df_list_test, type_d='test', method='difference')
y_test = df_list_test['time_to_eruption'].values.reshape(-1,1)

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
#norm train dataset
train_mean = X_train.mean()
train_sigma = X_train.std()
train_norm = (X_train - train_mean) / train_sigma

#scale
x_train_max = train_norm.max()
x_train_min = train_norm.min()
X_train_std = (train_norm - x_train_min) / (x_train_max - x_train_min)

y_train_max = y_train.max()
y_train_min = y_train.min()
y_train_std = (y_train - y_train_min) / (y_train_max - y_train_min)
#y_train_scaled = y_train_std * (y_train_max - y_train_min) + y_train_min

#Apply train mean, std to the test dataset
#norm test dataset
test_norm = (X_test - train_mean) / train_sigma
#scale
x_test_max = test_norm.max()
x_test_min = test_norm.min()
X_test_std = (test_norm - x_test_min) / (x_test_max - x_test_min)
y_test_std = y_test

In [ ]:
train_mean, train_sigma, x_train_max, x_train_min, y_train_max,y_train_min, x_test_max,x_test_min

In [ ]:
X_train_std.max(), X_train_std.min(), y_train_std.max(), y_train_std.min(), X_test_std.max(), X_test_std.min()

In [ ]:
df_list_train.shape, y_train.shape, X_train_std.shape, y_train.shape, X_test_std.shape, y_test.shape 

In [ ]:
# Plot some training and test images
fig, ax = plt.subplots(1,4, figsize=(20,14))

#train
ax[0].imshow(X_train_std[0,:,:,0])
ax[0].set_title('Segment '+str(df_list_train['segment_id'].iloc[0]))
ax[1].imshow(X_train_std[1,:,:,0])
ax[1].set_title('Segment '+str(df_list_train['segment_id'].iloc[1]))
#test
ax[2].imshow(X_test_std[0,:,:,0])
ax[2].set_title('Segment '+str(df_list_test['segment_id'].iloc[0]))
ax[3].imshow(X_test_std[1,:,:,0])
ax[3].set_title('Segment '+str(df_list_test['segment_id'].iloc[1]))
plt.show()

del df_list_train, df_list_test
gc.collect()

In [ ]:
features = np.squeeze(X_train_std)
labels = np.squeeze(y_train_std)
X_test = np.squeeze(X_test_std)
y_test = np.squeeze(y_test)
        
print(features.shape, labels.shape, X_test.shape, y_test.shape)

In [ ]:
class INGV(Dataset):   
    def __init__(self, features=features, labels=labels, p_flip=0., k=8, **kwargs):
        self.a = None
        self.features = features
        self.labels = labels
        self.k = k
        self.p_flip = p_flip
        super().__init__(**kwargs)
    
    def read(self):
        self.a = _mnist_grid_graph(self.k)
        self.a = _flip_random_edges(self.a, self.p_flip)

        x = self.features.reshape(-1, IMG_SIZE**2, 1)
        y = self.labels
        
        return [Graph(x=x_, y=y_) for x_, y_ in zip(x, y)]

In [ ]:
def _grid_coordinates(side):
    M = side ** 2
    x = np.linspace(0, 1, side, dtype=np.float32)
    y = np.linspace(0, 1, side, dtype=np.float32)
    xx, yy = np.meshgrid(x, y)
    z = np.empty((M, 2), np.float32)
    z[:, 0] = xx.reshape(M)
    z[:, 1] = yy.reshape(M)
    return z


def _get_adj_from_data(X, k, **kwargs):
    A = kneighbors_graph(X, k, **kwargs).toarray()
    A = sp.csr_matrix(np.maximum(A, A.T))
    return A


def _mnist_grid_graph(k):
    X = _grid_coordinates(IMG_SIZE)
    A = _get_adj_from_data(X, k, 
                           mode='connectivity', 
                           metric='euclidean', 
                           include_self=False)
    return A


def _flip_random_edges(A, percent):
    if not A.shape[0] == A.shape[1]:
        raise ValueError('A must be a square matrix.')
    
    dtype = A.dtype
    A = sp.lil_matrix(A).astype(np.bool)
    
    n_elem = A.shape[0] ** 2
    n_elem_to_flip = round(percent * n_elem)
    unique_idx = np.random.choice(n_elem, replace=False, size=n_elem_to_flip)
    row_idx = unique_idx // A.shape[0]
    col_idx = unique_idx % A.shape[0]
    idxs = np.stack((row_idx, col_idx)).T
    
    for i in idxs:
        i = tuple(i)
        A[i] = np.logical_not(A[i])
    A = A.tocsr().astype(dtype)
    A.eliminate_zeros()
    return A

In [ ]:
# Build model
class Net(Model):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.conv1 = GCNConv(32, activation='elu', kernel_regularizer=l2(l2_reg))
        self.conv2 = GCNConv(32, activation='elu', kernel_regularizer=l2(l2_reg))
        self.flatten = Flatten()
        self.fc1 = Dense(512, activation='relu')  
        #self.fc2 = Dense(1, activation='linear')
        self.fc2 = Dense(1, activation='sigmoid') 
        
    def call(self, inputs):
        x, a = inputs
        x = self.conv1([x, a])
        x = self.conv2([x, a])
        output = self.flatten(x)
        output = self.fc1(output)
        output = self.fc2(output)

        return output

In [ ]:
# Create model

with strategy.scope():
    model = Net()
    optimizer = Adam(lr=0.0001) 
    loss_fn = MeanAbsoluteError()
    acc_fn = tf.keras.metrics.MeanAbsoluteError()    
    
    #model.compile(optimizer=optimizer,
    #              loss='mae', 
    #              metrics=['mae'])


In [ ]:
# Training function
@tf.function
def train_on_batch(inputs, target):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_fn(target, predictions) + sum(model.losses)
        acc = acc_fn(target, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss, acc


# Evaluation function
def evaluate(loader):
    step = 0
    results = []
    for batch in loader:
        step += 1
        x, target = batch
        predictions = model([x, adj], training=False)
        loss = loss_fn(target, predictions)
        acc = acc_fn(target, predictions)
        results.append((loss, acc, len(target)))  # Keep track of batch size
        if step == loader.steps_per_epoch:
            results = np.array(results)
            return np.average(results[:, :-1], 0, weights=results[:, -1])

In [ ]:
# The adjacency matrix is stored as an attribute of the dataset.
# Create filter for GCN and convert to sparse tensor.

data = INGV(features, labels, k=kk, p_flip=0 )

adj = data.a
adj = GCNConv.preprocess(adj)
adj = sp_matrix_to_sp_tensor(adj)

# Train/valid split
p_split = int(0.20 * len(data))  #20 percent split
np.random.shuffle(data)
data_tr, data_va = data[:-p_split], data[-p_split:]

len(data_tr), len(data_va)


In [ ]:
# Setup training# Setup training
best_val_loss = 99999
current_patience = patience
step = 0

# We can use PackedBatchLoader because we only need to create batches of node
# features with the same dimensions.
loader_tr = PackedBatchLoader(data_tr, batch_size=batch_size, epochs=epochs)
loader_va = PackedBatchLoader(data_va, batch_size=batch_size)

In [ ]:
# Training loop
results_tr = []

loss_history = []
epoch_cnt = 0
epoch_mod =1

if (100 < epochs < 1000):
    epoch_mod=10
elif (epochs > 1000):
    epoch_mod=20

for batch in loader_tr:
    step += 1

    # Training step
    x, y = batch                              #x(batch_size, IMG**2), y(batch_size,)
    #l, a = model.train_on_batch([x, adj], y) #TPUs do not support DT_VARIANT
    l, a = train_on_batch([x, adj], y)        #l=loss, a=acc
    results_tr.append((l, a, len(y)))

    if step == loader_tr.steps_per_epoch:
        results_va = evaluate(loader_va)      #Validate
        if results_va[0] < best_val_loss:
            best_val_loss = results_va[0]
            current_patience = patience
            #results_te = evaluate(loader_te)
        else:
            current_patience -= 1
            if current_patience == 0:
                print('Early stopping')
                break

        # Print results
        results_tr = np.array(results_tr)
        results_tr = np.average(results_tr[:, :-1], 0, weights=results_tr[:, -1])  
        MAE = results_va[1] * (y_train_max - y_train_min) + y_train_min
        epoch_cnt +=1
        
        if (epoch_cnt%epoch_mod ==0):               
            print('Epoch {:3}/{:<3} | '
                  'Train loss: {:.5f}, mae: {:.5f} | '
                  'Val loss: {:.5f}, mae: {:.5f} | '
                  'MAE: {:10.1f}'
                  .format(epochs, epoch_cnt, *results_tr, *results_va, MAE))
        
        loss_history.append((results_tr[0], results_tr[1], 
                             results_va[0], results_va[1]))
        
        # Reset epoch
        results_tr = []
        step = 0
        
loss_history = np.array(loss_history)         

In [ ]:
model.summary()

In [ ]:
# Plots
plt.figure(figsize=(10, 5))

plt.subplot(121)
plt.plot(loss_history[:, 0], label='Train loss')
plt.plot(loss_history[:, 2], label='Val loss')
plt.legend()
plt.ylabel('Loss')
plt.xlabel('Epoch')

plt.subplot(122)
plt.plot(loss_history[:, 1], label='Train acc')
plt.plot(loss_history[:, 3], label='Val acc')
plt.legend()
plt.ylabel('Acc')
plt.xlabel('Epoch')

plt.show()

In [ ]:
print('Validate model')
y_pred=[]
y_true=[]

loader_va = PackedBatchLoader(data_va, batch_size=batch_size, epochs=1)
#batches = [b for b in loader_va]
#x,y = batches[-1]

for batch in loader_va:
    x, y = batch   
    p_va = model([x, adj], training=False)  #predict label per batch  
    y_pred.append(p_va)   
    y_va = np.vstack(y)   #True label per batch
    y_true.append(y_va)

In [ ]:
x.shape, y.shape, np.shape(y_true),np.shape(y_pred)

In [ ]:
#Last batch of Validate images

val_images = np.squeeze(x)     #(batch,IMG_SIZE**2) <- (batch,IMG_SIZE**2,1)
val_images = np.reshape(val_images,[val_images.shape[0], IMG_SIZE,-1])

y_va = np.squeeze(y_va)
p_va = np.squeeze(p_va)

num_plt = int(np.sqrt(len(val_images)))
num_plt = np.amin([5,num_plt])
num_fig = num_plt**2

plt.figure(figsize=(13,14))
for i in range(num_fig):
    plt.subplot(num_plt,num_plt,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(val_images[i], cmap=plt.cm.binary)
    plt.xlabel(p_va[i]) #Predict label
    plt.title(y_va[i])  #True label
plt.show()

In [ ]:
print('Testing model')

data_testset = INGV(X_test_std, y_test, k=kk, p_flip=0 )

adj_tst = data_testset.a
adj_tst = GCNConv.preprocess(adj_tst)
adj_tst = sp_matrix_to_sp_tensor(adj_tst)

data_te = data_testset
len(data_te)

In [ ]:
#loader_te = PackedBatchLoader(data_te, batch_size=data_te.n_graphs, epochs=1)
loader_te = PackedBatchLoader(data_te, batch_size=batch_size, epochs=1)

y_pred=[]
for batch in loader_te:
    x_te, y_te = batch    
    p_te = model([x_te, adj_tst], training=False)    
    y_pred.append(p_te)

In [ ]:
#Plot last batch of Test images

test_images = np.squeeze(x_te)
test_images = np.reshape(test_images,[test_images.shape[0], IMG_SIZE,-1])
p_te = np.squeeze(p_te)

#Plot Test images
num_plt = int(np.sqrt(len(test_images)))
num_plt = np.amin([5,num_plt])
num_fig = num_plt**2

plt.figure(figsize=(10,10))
for i in range(num_fig):
    plt.subplot(num_plt,num_plt,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(test_images[i], cmap=plt.cm.binary)
    plt.xlabel(p_te[i])
plt.show()

**Submission:**

In [ ]:
flat_list=[item for sublist in y_pred for item in sublist]       
flat_list= np.squeeze(flat_list)
np.shape(flat_list)

In [ ]:
y_pred_scaled = flat_list * (y_train_max - y_train_min) + y_train_min
len(y_pred_scaled)

In [ ]:
sub.head(3)

In [ ]:
sub.shape, y_pred_scaled.shape 

In [ ]:
if not DEBUG:
    sub['time_to_eruption'] = y_pred_scaled
    sub.to_csv('submission.csv', header=True, index=False)
    !cat submission.csv